In [3]:

import numpy as np
import scipy.integrate
import scipy.optimize
from bokeh.plotting import figure, output_file, show

import bokeh.plotting
import bokeh.io
from bokeh.models import Span


# We will use Matplotlib to make a 3D plot
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import panel as pn
pn.extension()

bokeh.io.output_notebook()
print(1)


Loading BokehJS ...

1


In [4]:
def Self_regenerate_model(molecules,t,alpha,beta,K,KTL,KTX,lamb1,lamb2):
    R,dT,dG,mT,mG,pT,pG,TX,TL = molecules
    return np.array(
        [
            -alpha*TX*dT/(dT+dG+KTX)*pT*R/(R+K)-alpha*TX*dG/(dT+dG+KTX)*pT*R/(R+K)-beta*TL*mT/(mT+mG+KTL)*R/(R+K)-beta*TL*mG/(mT+mG+KTL)*R/(R+K),
            0,
            0,
            alpha*TX*dT/(dT+dG+KTX)*pT*R/(R+K),
            alpha*TX*dG/(dT+dG+KTX)*pT*R/(R+K),
            beta*TL*(mT/(mT+mG+KTL))*(R/(R+K)),
            beta*TL*mG/(mT+mG+KTL)*R/(R+K),
            -lamb1*TX,
            -lamb1*TL
        ]
    )
    

In [9]:
#index 

idx_R =0
idx_dT =1
idx_dG =2
idx_mT =3
idx_mG =4
idx_pT =5
idx_pG =6
idx_TX =7
idx_TL =8


#common parameters 
alpha=0.7;
beta=0.07;
K=1.0;
KTL=10;
KTX=1;
lamb1=0.0001;
lamb2=0.0001;
DR = 0.2

#initial condition
R0=100.0;
dT0=50;
dG0=1;
mT0=0.0;
mG0=0.0;
pT0=1;
pG0=0.0;
TX0=1;
TL0=1;

molecules0 = [R0,dT0,dG0,mT0,mG0,pT0,pG0,TX0,TL0]




#stage 1 kick start

molecules0 = [R0,dT0,dG0,mT0,mG0,pT0,pG0,TX0,TL0]
result_all=molecules0

for n in range (1,12):

    #define time
    t_start= n*15
    t_end = (n+1)*15
    t= np.linspace(t_start,t_end,2)

    #solve equation and save result
    result = scipy.integrate.odeint(Self_regenerate_model, molecules0, t, args=(alpha,beta,K,KTL,KTX,lamb1,lamb2))
    result_all = np.append(result_all,result[1])
    #update parameter
    molecules0 = result.transpose()[:,-1]
    
    #dilution 
    ###replenish 
    molecules0[idx_R] = R0*DR+(1-DR)*molecules0[idx_R]
    molecules0[idx_pT] =pT0*DR+(1-DR)*molecules0[idx_pT]

    ###diute out
    molecules0[idx_mT] = (1-DR)*molecules0[idx_mT] 
    molecules0[idx_mG] = (1-DR)*molecules0[idx_mG]
    molecules0[idx_pG] = (1-DR)*molecules0[idx_pG]



#stage 2 self regeneration 

for n in range (12,64):

    #define time
    t_start= n*15
    t_end = (n+1)*15
    t= np.linspace(t_start,t_end,2)

    #solve equation and save result
    result = scipy.integrate.odeint(Self_regenerate_model, molecules0, t, args=(alpha,beta,K,KTL,KTX,lamb1,lamb2))
    result_all = np.append(result_all,result[1])
    #update parameter
    molecules0 = result.transpose()[:,-1]
    
    #dilution 
    ###replenish 
    molecules0[idx_R] = R0*DR+(1-DR)*molecules0[idx_R]
    

    ###diute out
    molecules0[idx_mT] = (1-DR)*molecules0[idx_mT] 
    molecules0[idx_mG] = (1-DR)*molecules0[idx_mG]
    molecules0[idx_pT] = (1-DR)*molecules0[idx_pT] 
    molecules0[idx_pG] = (1-DR)*molecules0[idx_pG]


for n in range (64,96):

    #define time
    t_start= n*15
    t_end = (n+1)*15
    t= np.linspace(t_start,t_end,2)

    #solve equation and save result
    result = scipy.integrate.odeint(Self_regenerate_model, molecules0, t, args=(alpha,beta,K,KTL,KTX,lamb1,lamb2))
    result_all = np.append(result_all,result[1])
    #update parameter
    molecules0 = result.transpose()[:,-1]
    
    #dilution 
    ###replenish 
    molecules0[idx_R] = R0*DR+(1-DR)*molecules0[idx_R]

    ###diute out
    molecules0[idx_mT] = (1-DR)*molecules0[idx_mT] 
    molecules0[idx_mG] = (1-DR)*molecules0[idx_mG]
    molecules0[idx_pT] = (1-DR)*molecules0[idx_pT] 
    molecules0[idx_pG] = (1-DR)*molecules0[idx_pG]
    molecules0[idx_dT] = (1-DR)*molecules0[idx_dT]
    molecules0[idx_dG] = (1-DR)*molecules0[idx_dG]


result_all = np.resize(result_all,(96,9))
R,dT,dG,mT,mG,pT,pG,TX,TL = result_all.transpose()




In [13]:
t = np.linspace(0, 1440, 96)
p = bokeh.plotting.figure(
    plot_width=800,
    plot_height=400,
    x_axis_label="t",
    y_axis_label="dimensionless concentration",
    y_axis_type="linear",

)

colors = bokeh.palettes.d3["Category10"][3]

# Populate glyphs
p.line(
    t/60, pG, line_width=2, color=colors[0]
)
vline1 = Span(location=3, dimension='height', line_color='black', line_width=1,line_dash='dashed')
vline2 = Span(location=16, dimension='height', line_color='black', line_width=1,line_dash='dashed')
p.add_layout(vline1)
p.add_layout(vline2)

show(p)
